# The MuJoCo `dexterity` Suite

**Note:** A GPU runtime is required to run the notebook. You can switch to one by clicking the `Runtime` menu in the toolbar and selecting `Change runtime type`.

### Installing `dexterity` on Colab

In [ ]:
import subprocess

if subprocess.run("nvidia-smi").returncode:
    raise RuntimeError(
        "Cannot communicate with GPU. "
        "Make sure you are using a GPU Colab runtime. "
        "Go to the Runtime menu and select Choose runtime type."
    )

print("Installing dexterity...")
!pip install -q --upgrade dexterity

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

### Imports

In [ ]:
from dexterity import manipulation
import dm_env
import numpy as np
from PIL import Image

In [ ]:
# Print available domains and tasks.
for domain, tasks in manipulation.TASKS_BY_DOMAIN.items():
    print(domain)
    for task in tasks:
        print(f"\t{task}")

In [ ]:
env = manipulation.load(domain_name="roller", task_name="state_dense", seed=0)

In [ ]:
action_spec = env.action_spec()
print(action_spec)

In [ ]:
obs_spec = env.observation_spec()
print(obs_spec)

In [ ]:
# get angle and axis
env.task.goal_generator._sampler.axis

In [ ]:

timestep = env.reset()

In [ ]:
pretty(timestep.observation)

In [ ]:
Image.fromarray(env.physics.render(camera_id="front_close"))

In [ ]:
def random_policy(timestep: dm_env.TimeStep) -> np.ndarray:
    del timestep  # Unused
    action = np.random.uniform(action_spec.minimum, action_spec.maximum)
    return action.astype(action_spec.dtype)

In [ ]:
def change_pitch_policy(timestep: dm_env.TimeStep) -> np.ndarray:
    action = np.zeros(action_spec.shape)
    action[3] = 0.02
    return action.astype(action_spec.dtype)

In [ ]:
def pretty(d, indent=0):
  for key, value in d.items():
    print('\t' * indent + str(key))
    if isinstance(value, dict):
      pretty(value, indent + 1)
    else:
      print('\t' * (indent + 1) + str(value))


In [ ]:
action = change_pitch_policy(timestep)
action = np.array([0,0,0,0.05])
for _ in range(1):
  timestep = env.step(action)


In [ ]:
Image.fromarray(env.render(mode="rgb_array"))

In [ ]:
print(timestep)

In [ ]:
from dexterity.utils.dm2gym import GymEnv
from dexterity import manipulation

env = GymEnv(domain_name="roller", task_name="state_dense")